# This notebook aims to explore aligning RAW images

We want to compare aligning on a per channel basis and aligning a de-mosaiced images

In [ ]:
import cv2
import numpy as np
import rawpy
import matplotlib.pyplot as plt
import os

raw_dir = os.path.join('.', 'first-dataset', 'first-dataset-RAW')


def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)

    im = ((im - im.min()) / (im.max() - im.min()) * 255).astype(np.uint8)
    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2, 0:W:2, :],
                          im[0:H:2, 1:W:2, :],
                          im[1:H:2, 1:W:2, :],
                          im[1:H:2, 0:W:2, :]), axis=2)
    return out

def align_images(img1, img2):
    """Aligns img2 to img1 using ORB feature matching and RANSAC."""
    orb = cv2.ORB_create(5000)

    # Detect keypoints and descriptors
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)

    # Match features using BFMatcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)  # Sort by distance

    # Use RANSAC to find homography
    if len(matches) > 10:
        src_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        
        # Warp img2 to align with img1 (preserving colors)
        aligned_img = cv2.warpPerspective(img2, H, (img1.shape[1], img1.shape[0]))
        
        return aligned_img, H
    else:
        raise ValueError("Not enough matches found.")

def grayscale_from_raw(raw):

    rgb = raw.postprocess()  # Process RAW file to RGB
    # image = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
    gray_downscaled = cv2.resize(gray, (gray.shape[1] // 2, gray.shape[0] // 2), interpolation=cv2.INTER_AREA)
    return  gray_downscaled

def homography_error_matrix(homographies):
    """
    Given a list of 3x3 homography matrices, computes an NxN error matrix where
    error_matrix[i, j] is the Frobenius norm of (I - H_j^-1 * H_i).

    :param homographies: List of N homography matrices (each 3x3).
    :return: NxN numpy array of error values.
    """
    N = len(homographies)
    error_mat = np.zeros((N, N), dtype=np.float64)
    
    for i in range(N):
        H_i = homographies[i]
        for j in range(N):
            H_j = homographies[j]
            
            # Attempt to invert H_j. If singular, treat the error as infinite.
            try:
                H_j_inv = np.linalg.inv(H_j)
            except np.linalg.LinAlgError:
                error_mat[i, j] = np.inf
                continue
            
            # Combined transform from j to i
            H_combined = np.dot(H_j_inv, H_i)
            
            # Measure difference from the identity
            diff = np.eye(3) - H_combined
            error_mat[i, j] = np.linalg.norm(diff, ord='fro')
    
    return error_mat

# Calculate projections
with rawpy.imread(os.path.join(raw_dir, 'IMG_7781.CR2')) as raw1, \
     rawpy.imread(os.path.join(raw_dir, 'IMG_7782.CR2')) as raw2:
    
    packed1 = pack_raw(raw1)    
    packed1 = np.rot90(packed1, k=1)
    packed1R, packed1G1, packed1B, packed1G2 = packed1[:,:,0], packed1[:,:,1], packed1[:,:,2], packed1[:,:,3]
    
    packed2 = pack_raw(raw2)
    packed2 = np.rot90(packed2, k=1)
    packed2R, packed2G1, packed2B, packed2G2 = packed2[:,:,0], packed2[:,:,1], packed2[:,:,2], packed2[:,:,3]

    pairs = [(packed1R, packed2R), (packed1G1, packed2G1), (packed1B, packed2B), (packed1G2, packed2G2)]
    aligned_images = []
    projections = []
    # Calculate projections for each color channel individually
    for img1, img2 in pairs:
        aligned, projection = align_images(img1, img2)    
        aligned_images.append(aligned)
        projections.append(projection)

    # Calculate projections for the processed combined grayscale image
    img1_grayscale = grayscale_from_raw(raw1)
    img2_grayscale = grayscale_from_raw(raw2)
    aligned_grayscale, projection_grayscale = align_images(img1_grayscale, img2_grayscale)


    for image in aligned_images:
        print(image.shape)
    for projection in projections:
        print(projection)
    print(projection_grayscale)

    projections.append(projection_grayscale)

    print("Error matrix. Shows the difference between the individual projection matrices. ")
    print(np.array2string(homography_error_matrix(projections), formatter={'float_kind':lambda x: f"{x:.3g}"}))
    


# with rawpy.imread(os.path.join(raw_dir, 'IMG_7781.CR2')) as raw1, \
#      rawpy.imread(os.path.join(raw_dir, 'IMG_7782.CR2')) as raw2:

#     packed1 = pack_raw(raw1)
#     packed1 = np.rot90(packed1, k=1)

#     packed2 = pack_raw(raw2)
#     packed2 = np.rot90(packed2, k=1)

#     fig, ax = plt.subplots(2,4, figsize=(15,10), sharey=True)
#     for i in range(4):
#         im = ax[0][i].imshow(packed1[:,:,i], cmap='gray')
#         ax[0][i].set_title(raw1.color_desc.decode('utf-8')[i])
#         if i == 0:
#             fig.colorbar(im, ax = ax.ravel())
#     for i in range(4):
#         im = ax[1][i].imshow(packed2[:,:,i], cmap='gray')
            
#     plt.show()
#     print()

(2145, 1428)
(2145, 1428)
(2145, 1428)
(2145, 1428)
[[ 1.67385787e+00 -5.83304867e-02 -6.17867741e+02]
 [ 5.87296908e-02  1.64115670e+00 -4.22199707e+02]
 [ 1.36236835e-05 -2.32549934e-05  1.00000000e+00]]
[[ 1.67052291e+00 -5.71598881e-02 -6.17779559e+02]
 [ 5.56710209e-02  1.64209427e+00 -4.23084807e+02]
 [ 1.10493992e-05 -2.27729496e-05  1.00000000e+00]]
[[ 1.66961144e+00 -5.68198205e-02 -6.17732604e+02]
 [ 5.53275621e-02  1.64106186e+00 -4.23016544e+02]
 [ 1.07290348e-05 -2.34352542e-05  1.00000000e+00]]
[[ 1.66310321e+00 -5.64436020e-02 -6.14835003e+02]
 [ 5.42914704e-02  1.63160306e+00 -4.15948729e+02]
 [ 1.00099297e-05 -2.56350095e-05  1.00000000e+00]]
[[ 1.66573451e+00 -5.88251624e-02 -6.13936252e+02]
 [ 5.30479972e-02  1.63738040e+00 -4.19070821e+02]
 [ 9.16736331e-06 -2.34671862e-05  1.00000000e+00]]
[[8.04e-14 0.544 0.506 4.27 3.05]
 [0.544 2.84e-14 0.0505 4.75 3.38]
 [0.506 0.0504 5.68e-14 4.7 3.33]
 [4.24 4.72 4.67 5.68e-14 1.99]
 [3.03 3.37 3.32 1.99 6.96e-14]]


The result here shows that the first R G and B projections are very close to each other. The other two matrixes. The last green one and the processed grayscale are slightly different. 